In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy
import pickle
UP = 0
LEFT = 1
OK = 2
batch_size = 100
K = 0
def read_user_rating_records():
    col_names = ['user_id', 'item_id', 'rating', 'timestamp']
    data_records = pd.read_csv(dir_path + rating_file, sep=',', names=col_names, engine='python')
    return data_records
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f)
def remove_infrequent_items(data, min_counts=20):
    df = deepcopy(data)
    counts = df['item_id'].value_counts()
    df = df[df["item_id"].isin(counts[counts >= min_counts].index)]

    print("items with < {} interactoins are removed".format(min_counts))
    # print(df.describe())
    return df

def remove_infrequent_users(data, min_counts=5):
    df = deepcopy(data)
    counts = df['user_id'].value_counts()
    df = df[df["user_id"].isin(counts[counts >= min_counts].index)]

    print("users with < {} interactoins are removed".format(min_counts))
    # print(df.describe())
    return df
def convert_data(data):
    # for each user, sort by timestamps
    df = deepcopy(data)
    df_ordered = df.sort_values(['timestamp'], ascending=True)
    data = df_ordered.groupby('user_id')['item_id'].apply(list)
    #print(data)
    #time_l = df_ordered.groupby('user')['checkin_time'].apply(list)
    #print(time_l)
    print("succressfully created sequencial data! head:", data.head(5))
    unique_data = df_ordered.groupby('user_id')['item_id'].nunique()
    data = data[unique_data[unique_data >= 10].index]
    print(data[:10])
    print(len(data))
    return data
def seqence_similarity(seq1,seq2):
    len1 = len(seq1)
    len2 = len(seq2)
    dp = np.zeros((len1+1,len2+1))
    for i in range(len1 + 1):
        dp[i][0]=i
    for j in range(len2 + 1):
        dp[0][j]=j
    for i in range(1,len1+1):
        for j in range(1,len2+1):
            delta=0 if seq1[i-1]==seq2[j-1] else 1
            dp[i][j] = min(dp[i - 1][j - 1] + delta, min(dp[i-1][j] + 1, dp[i][j - 1] + 1))
    length = max(len1,len2)
    distance = dp[len1][len2]
    similarity = (length-distance)/distance
    return similarity
def LCS(seq1,seq2):
    if (set(seq1).intersection(set(seq2)))==set():
        return []
    m = [[0 for x in range(len(seq2)+1)] for y in range(len(seq1)+1)]
    d = [[None for x in range(len(seq2)+1)] for y in range(len(seq1)+1)]
    for p1 in range(len(seq1)):
        for p2 in range(len(seq2)):
            if seq1[p1] == seq2[p2]:
                m[p1+1][p2+1] = m[p1][p2]+1
                d[p1+1][p2+1] = OK
            elif m[p1+1][p2]>m[p1][p2+1]:
                m[p1+1][p2+1]=m[p1+1][p2]
                d[p1+1][p2+1] = LEFT
            else:
                m[p1+1][p2+1]=m[p1][p2+1]
                d[p1+1][p2+1] = UP
    (p1,p2) = (len(seq1),len(seq2))
    index = []
    while m[p1][p2]:
        direction = d[p1][p2]
        #print(p1, p2)
        #print(direction)
        if direction == OK:
            index.append((p1-1,p2-1))
            p1 -= 1
            p2 -= 1
        elif direction == LEFT:
            p2 -= 1
        elif direction == UP:
            p1 -= 1
    index.reverse()
    return index


def merging(seq1,seq2,index):
    index_first = [x[0] for x in index]
    index_second = [x[1] for x in index]
    new_seq = []
    parts = len(index)
    p1 = 0
    p2 = 0
    for i in range(parts):
        idx1 = index_first[i]
        idx2 = index_second[i]
        new_seq+=seq1[p1:idx1]+seq2[p2:idx2]+[seq1[idx1]]
        p1 = idx1+1
        p2 = idx2+1

    new_seq+=seq1[p1:len(seq1)]+seq2[p2:len(seq2)]
    return new_seq


dir_path = './'
rating_file = 'ratings_Musical_Instruments.csv'

data_records = read_user_rating_records()
data_records.loc[data_records.rating < 4, 'rating'] = 0
data_records.loc[data_records.rating >= 4, 'rating'] = 1
data_records = data_records[data_records.rating > 0]
print(len(data_records['user_id'].unique()), len(data_records['item_id'].unique()))

filtered_data = remove_infrequent_users(data_records, 5)
filtered_data = remove_infrequent_items(filtered_data, 10)
print('num of users:{}, num of items:{}'.format(len(filtered_data['user_id'].unique()), len(filtered_data['item_id'].unique())))


seq_data = convert_data(filtered_data)
batch_num = len(seq_data)//batch_size
index = 0
series = []
for i in range(0,batch_num+1):
    print(index)
    end = index + 100 if index + 100 <= len(seq_data) else len(seq_data)
    series.append(seq_data[index:end])
    index += 100
merged_sequence = pd.Series()
for b in range(batch_num+1):
    for k in range(K):
        MAX = 0
        MAX_A,MAX_B = -1,-1
        for i  in range(len(series[b])):
            for j in range(i):
                #print(i,j)
                if i != j:
                    similarity = seqence_similarity(series[b][i],series[b][j])
                    if similarity>MAX:
                        MAX = similarity
                        MAX_A = i
                        MAX_B = j
                        print('update similarity, MAX S={} comes from {} and {}'.format(MAX,MAX_A,MAX_B))
        seq1 = series[b][MAX_A]
        seq2 = series[b][MAX_B]
        user_A = series[b].index[MAX_A]
        user_B = series[b].index[MAX_B]
        series[b] = series[b].drop([user_A,user_B])
        if len(seq1)<len(seq2):
            index = LCS(seq2,seq1)
            seq = merging(seq2,seq1,index)
            series[b][user_B]=seq
        else:
            index = LCS(seq1,seq2)
            seq = merging(seq1,seq2,index)
            series[b][user_A] = seq
        print('In batch {}, merging step {}, user {} and {} are merged'.format(b,k+1,user_A,user_B))
for b in range(batch_num+1):
    print(b)
    merged_sequence = pd.concat([merged_sequence,series[b]])
print(len(merged_sequence),len(seq_data))

278696 74603
users with < 5 interactoins are removed
items with < 10 interactoins are removed
num of users:5801, num of items:797
succressfully created sequencial data! head: user_id
A0028518312V9G9GA1SDL                             [B0001FTVD6, B0002MDTVS]
A00338543M2OZPUWO9ZRU     [B0002E1G5C, B0002D0CEO, B0002E1H9W, B0002IHFVM]
A00625243BI8W1SSZNLMD    [B000SAC5PA, B0009G1E0K, B000PO30QM, B0018TIAD...
A01458943UKX2HCM0VVN0     [B003NJF1G8, B001LU1SFO, B003VWJ2K8, B008GS3XLQ]
A04561483EYOO5BSJ3VEU                 [B005FKF1PY, B007T8CUNG, B009EOKTCM]
Name: item_id, dtype: object
user_id
A13IKQCJKFAP5S    [B000BUDO48, B002433RXI, B0009G1E0K, B0002GLDQ...
A15TYOEWBQYF0X    [B0002E2XCW, B0002GLDQM, B00BTGMI5O, B005PNXT6...
A15WZCSME5X74S    [B0002E1G5C, B003VWJ2K8, B003B01QL8, B002AQNGI...
A164BJ2NU1NSJZ    [B000PO30QM, B001L8NGJ2, B003AYRBWI, B0002E2GM...
A16Z3HTUIYPDH8    [B000B6DHAS, B0002GYW4C, B000RNB720, B0002CZSJ...
A1BH17V8BKLES     [B0002M6CVC, B000N5MK8M, B0002CZSJO, B0002D0CE.

In [10]:
print(len(merged_sequence[105]))
print(len(seq_data[105]))

10
10


In [12]:
user_item_dict = merged_sequence.to_dict()
user_mapping = []
item_set = set()
for user_id, item_list in merged_sequence.iteritems():
    user_mapping.append(user_id)
    for item_id in item_list:
        item_set.add(item_id)
item_mapping = list(item_set)

print(len(user_mapping), len(item_mapping))

107 589


In [21]:
user_item_dict = seq_data.to_dict()
user_mapping = []
item_set = set()
for user_id, item_list in seq_data.iteritems():
    user_mapping.append(user_id)
    for item_id in item_list:
        item_set.add(item_id)
item_mapping = list(item_set)

print(len(user_mapping), len(item_mapping))

107 589


In [22]:
def generate_inverse_mapping(data_list):
    inverse_mapping = dict()
    for inner_id, true_id in enumerate(data_list):
        inverse_mapping[true_id] = inner_id
    return inverse_mapping

def convert_to_inner_index(user_records, user_mapping, item_mapping):
    inner_user_records = []
    user_inverse_mapping = generate_inverse_mapping(user_mapping)
    item_inverse_mapping = generate_inverse_mapping(item_mapping)

    for user_id in range(len(user_mapping)):
        real_user_id = user_mapping[user_id]
        item_list = list(user_records[real_user_id])
        for index, real_item_id in enumerate(item_list):
            item_list[index] = item_inverse_mapping[real_item_id]
        inner_user_records.append(item_list)

    return inner_user_records, user_inverse_mapping, item_inverse_mapping

inner_data_records, user_inverse_mapping, item_inverse_mapping = convert_to_inner_index(user_item_dict, user_mapping, item_mapping)
print(inner_data_records[:5])

[[345, 51, 90, 117, 257, 290, 396, 19, 288, 185, 463, 25, 379, 587], [199, 117, 545, 384, 261, 428, 259, 459, 194, 313, 211, 185, 252, 499, 420, 579, 271, 90, 270, 393, 183, 326, 408, 260, 357], [428, 463, 224, 200, 113, 219, 448, 117, 286, 242], [290, 303, 529, 183, 305, 27, 360, 250, 35, 178], [324, 517, 585, 498, 387, 298, 251, 375, 75, 579]]


In [23]:
import pickle
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f)

In [25]:
save_obj(inner_data_records, '../data/dataset/Amazon/MI/MI_K={}_BS={}_item_sequences'.format(K,batch_size))
save_obj(user_mapping, '../data/dataset/Amazon/MI/MI_K={}_BS={}_user_mapping'.format(K,batch_size))
save_obj(item_mapping, '../data/dataset/Amazon/MI/MI_K={}_BS={}_item_mapping'.format(K,batch_size))

In [13]:
from scipy.sparse import csr_matrix

def generate_rating_matrix(train_set, num_users, num_items):
    # three lists are used to construct sparse matrix
    row = []
    col = []
    data = []
    for user_id, article_list in enumerate(train_set):
        for article in article_list:
            row.append(user_id)
            col.append(article)
            data.append(1)

    row = np.array(row)
    col = np.array(col)
    data = np.array(data)
    rating_matrix = csr_matrix((data, (row, col)), shape=(num_users, num_items))

    return rating_matrix

rating_matrix = generate_rating_matrix(inner_data_records, len(user_mapping), len(item_mapping))
rating_matrix = rating_matrix.transpose()

from sklearn.metrics.pairwise import cosine_similarity

relation_matrix = cosine_similarity(rating_matrix)

In [14]:
rating_matrix.nnz / float(len(user_mapping) * len(item_mapping))

0.08317669172932331

In [16]:
np.fill_diagonal(relation_matrix, 0)
max_count = 0
for i in range(len(item_mapping)):
    max_count = max(np.count_nonzero((relation_matrix[i] >= 0.2) == True), max_count)
    
print (max_count)

count = 0
for i in range(len(item_mapping)):
    if np.count_nonzero((relation_matrix[i] >= 0.2) == True) > 0:
        count += 1

print (count)
print (np.max(relation_matrix))
print (relation_matrix[0])
print (relation_matrix[-1])

70
152
1.0
[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.5        0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.4472136  0.
 0.         0.         0.         0.         0.         0.
 0.         0.37796447 0.70710678 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         

In [18]:
relation_matrix[relation_matrix < 0.2] = 0
relation_matrix[relation_matrix > 0] = 1
relation_matrix = csr_matrix(relation_matrix)
print(len(user_mapping), len(item_mapping))

28 152


In [ ]:
# process review content
import re
import nltk

# generate the whole document
all_review = []
for item_id in item_mapping:
    all_review.append([review_dict[item_id]])

# use nltk to remove stopwords, and stemming each word
from nltk.corpus import stopwords
stopwords_set = set(stopwords.words('english'))
porter_stemmer = nltk.PorterStemmer()

review_str = []
for i, movie in enumerate(all_review):
    # Use regular expressions to do a find-and-replace
    letters_only = re.sub("[^a-zA-Z]",  # The pattern to search for
                          " ",  # The pattern to replace it with
                          movie[0])  # The text to search
    # print letters_only

    letters_only = letters_only.lower()
    tokens = nltk.word_tokenize(letters_only)

    tokens = [w for w in tokens if w.lower() not in stopwords_set]
    # print tokens

    porter = [porter_stemmer.stem(t) for t in tokens]
    # print porter
    all_review[i] = porter
    review_str.append(' '.join(porter))

print review_str[-1]

In [ ]:
# convert to bag-of-words
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer="word", tokenizer=None, preprocessor=None, stop_words=None, min_df=3)
word_counts = vectorizer.fit_transform(review_str)
vocab = vectorizer.get_feature_names()

print len(vocab)
print word_counts.data.max()
print word_counts.data.min()
print len(item_mapping)

In [ ]:
rating_matrix.nnz / float(len(user_mapping) * len(item_mapping))

In [ ]:
# store bag-of-words to file
def vocabulary_to_file(vocab):
    f0 = open('vocabulary.txt', 'w')

    for word in vocab:
        f0.write(word + '\n')
    f0.close()


def word_count_to_file(item_list, word_count):
    f0 = open('word_counts.txt', 'w')
    for i, document in enumerate(word_count):
        indices = document.indices
        counts = document.data
        num_words = document.count_nonzero()

        f0.write(str(item_list[i]) + ' ' + str(num_words))
        for j, indice in enumerate(indices):
            f0.write(' ' + str(indice) + ':' + str(counts[j]))
        f0.write('\n')
    f0.close()

vocabulary_to_file(vocab)
word_count_to_file(item_mapping, word_counts)

In [ ]:
import pickle
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f)


In [ ]:
save_obj(user_item_dict, 'Books_user_records')
save_obj(user_mapping, 'Books_user_mapping')
save_obj(item_mapping, 'Books_item_mapping')
save_obj(relation_matrix, 'item_relation')

In [ ]:
print vocab[:10]
print all_review[-1]
print review_str[-1]

In [ ]:
word_to_index = dict()
for w_id, word in enumerate(vocab):
    word_to_index[word] = w_id

all_review_index = []
for i in range(len(review_str)):
    cur_review = review_str[i].split(' ')
    cur_index = []
    for word in cur_review:
        if word in word_to_index:
            cur_index.append(word_to_index[word])
    all_review_index.append(cur_index)
    
print all_review_index[-1]

In [ ]:
# store word sequence to a file
save_obj(all_review_index, 'review_word_sequence')

In [ ]:
print seq_data[-1]
user_inverse_mapping = generate_inverse_mapping(user_mapping)
item_inverse_mapping = generate_inverse_mapping(item_mapping)
print user_item_dict[user_mapping[-1]]
tmp = []
for item_id in seq_data[-1]:
    tmp.append(item_inverse_mapping[item_id])
print sorted(tmp)

In [ ]:
print all_review[-1]

In [ ]:
for i in range(word_counts.shape[0]):
    if word_counts.getrow(i).getnnz() == 0:
        print i